In [ ]:
import numpy as np
import os
import time
import mediapipe as mp
import h5py
from flask import Flask, render_template, Response
import cv2 
from tensorflow import keras
import pickle
import joblib
app = Flask(__name__,template_folder= 'MP_Data/templates')
model = loaded_model = pickle.load(open('random1200.pkl', 'rb'))
#model = keras.models.load_model('actionpro1.h5',compile = False)
#actions = np.array(['hello','bye','thanks', 'please','namaste','yes','no'])
#model = keras.models.load_model('actionhpn.h5')
actions = np.array(['thanks', 'please','namaste'])
label_map = {label:num for num, label in enumerate(actions)}
# Thirty videos worth of data

# Videos are going to be 30 frames in length
sequence_length = 30
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
@app.route('/', methods=["POST",'GET'])
def index():
    """Video streaming home page."""
    return render_template('index.html')
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

#colors = [(245,117,16), (117,245,16), (16,117,245),(245,117,16), (117,245,16), (16,117,245),(16,117,245)]
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame
cap = cv2.VideoCapture(0)
def gen():
  
  sequence = []
  sentence = []
  threshold = 0.98

  
  # Set mediapipe model 
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
      while cap.isOpened():

          # Read feed
          ret, frame = cap.read()
          print(frame.shape)  

          # Make detections
          image, results = mediapipe_detection(frame, holistic)
          
          # Draw landmarks
          draw_styled_landmarks(image, results)
          
          # 2. Prediction logic
          keypoints = extract_keypoints(results)

          sequence.append(keypoints)
          sequence = sequence[-30:]
          
           
          if len(sequence) == 30:
              #X = np.array(sequence)
              
              #X = np.array(sequence).reshape(1, (np.array(sequence).shape[0]*np.array(sequence).shape[1]))
              
             
              #res = model.predict(np.expand_dims(sequence, axis=0))[0]
              res = model.predict_proba(np.array(sequence).reshape(1, (np.array(sequence).shape[0]*np.array(sequence).shape[1])))[0]
              print(res)  
              print(actions[np.argmax(res)])
              
              
          #3. Viz logic
              if (res[0] > 0.5) or (res[1] > 0.96) or (res[2] > 0.7) : 
                  if len(sentence) > 0: 
                      if actions[np.argmax(res)] != sentence[-1]:
                          sentence.append(actions[np.argmax(res)])
                  else:
                      sentence.append(actions[np.argmax(res)])

              if len(sentence) > 5: 
                  sentence = sentence[-5:]

              # Viz probabilities
              image = prob_viz(res, actions, image, colors)
              
          cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
          cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
          
          # Show to screen
          #cv2.imshow('open_image',image)
          ret, buffer = cv2.imencode('.jpg', image)
          frame = buffer.tobytes()
          yield (b'--frame\r\n'
                 b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')        

                           

          # Break gracefully
          if cv2.waitKey(10) & 0xFF == ord('q'):
              break
      cap.release()
      cv2.destroyAllWindows()
        
def gen_frames():  
    while True:
        success, frame = cap.read()  # read the camera frame
        if not success:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')
    

if __name__ == "__main__":
	app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Aug/2021 14:36:18] "GET / HTTP/1.1" 200 -


(480, 640, 3)


127.0.0.1 - - [31/Aug/2021 14:36:18] "GET /video_feed HTTP/1.1" 200 -


(480, 640, 3)
(480, 640, 3)
(480, 640, 3)


127.0.0.1 - - [31/Aug/2021 14:36:19] "GET /static/thank_you.jpg HTTP/1.1" 304 -
127.0.0.1 - - [31/Aug/2021 14:36:19] "GET /static/please.png HTTP/1.1" 304 -
127.0.0.1 - - [31/Aug/2021 14:36:19] "GET /static/namaste.jpg HTTP/1.1" 304 -


(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
(480, 640, 3)
[0.16 0.74 0.1 ]
please
(480, 640, 3)
[0.16 0.75 0.09]
please
(480, 640, 3)
[0.18 0.75 0.07]
please
(480, 640, 3)
[0.19 0.74 0.07]
please
(480, 640, 3)
[0.19 0.74 0.07]
please
(480, 640, 3)
[0.18 0.74 0.08]
please
(480, 640, 3)
[0.17 0.77 0.06]
please
(480, 640, 3)
[0.16 0.79 0.05]
please
(480, 640, 3)
[0.14 0.82 0.04]
please
(480, 640, 3)
[0.13 0.83 0.04]
please
(480, 640, 3)
[0.1  0.86 0.04]
please
(480, 640, 3)
[0.08 0.88 0.04]
please
(480, 640, 3)
[0.04 0.93 0.03]
please
(480, 640, 3)
[0.01 0.97 0.02]
please
(480, 640, 3)
[0.01 0.97 0.02]
please
(480, 640, 3)
[0.01 0.97 0.02]
please
(480, 640, 3)
[0.   0.98 0.02]
please
(480

[0.65 0.07 0.28]
thanks
(480, 640, 3)
[0.61 0.11 0.28]
thanks
(480, 640, 3)
[0.64 0.1  0.26]
thanks
(480, 640, 3)
[0.7  0.06 0.24]
thanks
(480, 640, 3)
[0.67 0.1  0.23]
thanks
(480, 640, 3)
[0.61 0.09 0.3 ]
thanks
(480, 640, 3)
[0.72 0.07 0.21]
thanks
(480, 640, 3)
[0.69 0.08 0.23]
thanks
(480, 640, 3)
[0.6 0.1 0.3]
thanks
(480, 640, 3)
[0.67 0.07 0.26]
thanks
(480, 640, 3)
[0.59 0.11 0.3 ]
thanks
(480, 640, 3)
[0.65 0.11 0.24]
thanks
(480, 640, 3)
[0.61 0.11 0.28]
thanks
(480, 640, 3)
[0.6  0.15 0.25]
thanks
(480, 640, 3)
[0.65 0.07 0.28]
thanks
(480, 640, 3)
[0.7  0.04 0.26]
thanks
(480, 640, 3)
[0.67 0.05 0.28]
thanks
(480, 640, 3)
[0.64 0.06 0.3 ]
thanks
(480, 640, 3)
[0.55 0.1  0.35]
thanks
(480, 640, 3)
[0.61 0.1  0.29]
thanks
(480, 640, 3)
[0.58 0.1  0.32]
thanks
(480, 640, 3)
[0.57 0.14 0.29]
thanks
(480, 640, 3)
[0.57 0.12 0.31]
thanks
(480, 640, 3)
[0.58 0.07 0.35]
thanks
(480, 640, 3)
[0.62 0.04 0.34]
thanks
(480, 640, 3)
[0.55 0.07 0.38]
thanks
(480, 640, 3)
[0.49 0.12 0.39

[0.22 0.05 0.73]
namaste
(480, 640, 3)
[0.19 0.06 0.75]
namaste
(480, 640, 3)
[0.17 0.04 0.79]
namaste
(480, 640, 3)
[0.21 0.06 0.73]
namaste
(480, 640, 3)
[0.23 0.07 0.7 ]
namaste
(480, 640, 3)
[0.21 0.1  0.69]
namaste
(480, 640, 3)
[0.2  0.09 0.71]
namaste
(480, 640, 3)
[0.2  0.06 0.74]
namaste
(480, 640, 3)
[0.25 0.05 0.7 ]
namaste
(480, 640, 3)
[0.24 0.11 0.65]
namaste
(480, 640, 3)
[0.25 0.11 0.64]
namaste
(480, 640, 3)
[0.24 0.05 0.71]
namaste
(480, 640, 3)
[0.21 0.09 0.7 ]
namaste
(480, 640, 3)
[0.22 0.11 0.67]
namaste
(480, 640, 3)
[0.22 0.15 0.63]
namaste
(480, 640, 3)
[0.22 0.16 0.62]
namaste
(480, 640, 3)
[0.16 0.11 0.73]
namaste
(480, 640, 3)
[0.2  0.08 0.72]
namaste
(480, 640, 3)
[0.19 0.1  0.71]
namaste
(480, 640, 3)
[0.2  0.12 0.68]
namaste
(480, 640, 3)
[0.28 0.11 0.61]
namaste
(480, 640, 3)
[0.25 0.05 0.7 ]
namaste
(480, 640, 3)
[0.28 0.07 0.65]
namaste
(480, 640, 3)
[0.35 0.07 0.58]
namaste
(480, 640, 3)
[0.28 0.06 0.66]
namaste
(480, 640, 3)
[0.35 0.07 0.58]
namaste


[0.01 0.94 0.05]
please
(480, 640, 3)
[0.01 0.94 0.05]
please
(480, 640, 3)
[0.01 0.94 0.05]
please
(480, 640, 3)
[0.01 0.94 0.05]
please
(480, 640, 3)
[0.02 0.93 0.05]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.01 0.94 0.05]
please
(480, 640, 3)
[0.02 0.93 0.05]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.93 0.05]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.91 0.07]
please
(480, 640, 3)
[0.02 0.92 0.06]
please
(480, 640, 3)
[0.02 0.91 0.07]
please
(480, 640, 3)
[0.02 0.91 0.07]
please
(480, 640, 3)
[0.02 0.9  0.08]
please
(480, 640, 3)
[0.02 0.91 0.07]
please
(480, 640, 3)
[0.02 0.9  0.08]
please
(480, 640, 3)
[0.02 0.91 0.07]
please
(480, 640, 3)
[0.02 0.9  0.08]
please
(480, 640, 3)
[0.03 0.9  0

In [ ]:
print("hello")